### Q. 로이터 뉴스를 46개의 상호 배타적인 토픽으로 분류하는 신경망 모델을 개발 및 평가하세요.
- 데이터 소스 : from keras.datasets import reuters
- 데이터 및 레이블 벡터 변환 : 주어진 사용자 함수 참조
- 모델 구성 : 2개 은닉층(64개 유닛), 1개 출력층
- 성능 개선 : 과대 적합 방지 3개 방법 적용
- 시각화 : Accuracy, Val_Accuracy 비교 시사점 도출
#### 공용 폴더에 제출하세요.(401공유폴더\NLP)
- 파일명 : 이름_평가Score.ipynb

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras import models
from keras import layers
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint

# 데이터 소스
from keras.datasets import reuters

# 랜덤 시드
np.random.seed(0)

number_of_features = 10000

# 데이터 로드 후 분류
data = reuters.load_data(num_words=number_of_features)
# (data_train, target_vector_train), (data_test, target_vector_test) = data
(x_train, y_train), (x_test, y_test) = data


# x 데이터 원핫인코딩
tokenizer = Tokenizer(num_words=number_of_features)
train_data = tokenizer.sequences_to_matrix(x_train, mode="binary")
test_data = tokenizer.sequences_to_matrix(x_test, mode="binary")


# y 데이터 원핫인코딩
train_target = to_categorical(y_train)
test_target = to_categorical(y_test)


# 모델 생성
model = models.Sequential()
model.add(layers.Dropout(0.2, input_shape=(number_of_features,)))

model.add(layers.Dense(units=64
                         # 가중치 규제 설정
                         , kernel_regularizer=regularizers.l2(0.01)
                         , activation="relu"
                         , input_shape=(number_of_features,)))

model.add(layers.Dropout(0.3))

model.add(layers.Dense(units=64
                         # 가중치 규제 설정
                         #, kernel_regularizer=regularizers.l2(0.01)
                         , activation="relu"))

model.add(layers.Dropout(0.1))

model.add(layers.Dense(units=46, activation="softmax"))

# 모델 컴파일
model.compile(loss="categorical_crossentropy"
                , optimizer="rmsprop"
                , metrics=["accuracy"])

# 조기 종료  설정
callbacks = [EarlyStopping(monitor="val_loss", patience=2)
             , ModelCheckpoint(filepath="bestm_model.h5", monitor="val_loss", save_best_only=True)]

# 모델 학습
hist = model.fit(
    train_data, train_target
    , epochs=20
    , verbose=1
    , batch_size=10000
    , validation_data=(test_data, test_target)
)

# 훈련&테스트 정확도와 손실
train_acc = hist.history["accuracy"]
test_acc = hist.history["val_accuracy"]

train_loss = hist.history['loss']
test_loss = hist.history['val_loss']

# 에포크 횟수를 사용해 카운트 객체를 만듭니다.
epoch_count = range(1, len(train_acc) + 1)

print('\n')
print('모델 성능 :')
score = model.evaluate(test_data, test_target)
print('accuracy= ', score[1], 'loss= ', score[0])

In [ ]:
# import matplotlib.pyplot as plt

# 정확도 시각화
plt.plot(epoch_count, train_acc, "bo", label="Accuracy")
plt.plot(epoch_count, test_acc, "b", label="Val_Accuracy")

plt.legend(["Accuracy", "Val_Accuracy"])
plt.xlabel("Epoch")
plt.ylabel("Accuracy Score")
plt.show()
plt.close()

In [ ]:
# import matplotlib.pyplot as plt

# 손실 시각화
plt.plot(epoch_count, train_loss, "bo", label='Loss')
plt.plot(epoch_count, test_loss, "b", label='Val_Loss')
plt.title('Train and Test loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()